# Teste com o pacote Plotly para plotagens estáticas e interativas
## Maiky Barreto da Silva

In [1]:
# Importando os módulos e o arquivo

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

In [2]:
# Importando os pacotes do plotly

import plotly.graph_objects as go
import plotly.express as px
from IPython.html.widgets import interact

# Importando os pacotes do ipywidgets

import ipywidgets as widgets
import IPython.display as display
import datetime

/home/maiky/.local/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [ ]:
# Importando o arquivo

df = pd.read_csv('https://bi.static.es.gov.br/covid19/MICRODADOS.csv', sep = ';', encoding = 'iso8859_15')

In [ ]:
# Pré-análise

df.DataNotificacao = df.DataNotificacao.astype(np.datetime64)
df.DataObito = df.DataObito.astype(np.datetime64)

# Análise nos casos no Estado

df_confirmados = df.query("Classificacao == 'Confirmados'")
dist_confirmados = df_confirmados.groupby('DataNotificacao').count().DataCadastro
rm_confirmados = df_confirmados.groupby('DataNotificacao').count().DataCadastro.rolling(14).mean()

# Análise dos mortos no Estado

df_obitos = df_confirmados.query("DataObito > '2020'")
total_obitos = df_obitos.pivot_table('DataNotificacao', 'DataObito', 'Classificacao', aggfunc='count')
rm_total_obitos = total_obitos.rolling(14).mean()

In [ ]:
# Análise dos casos na GV

gv = ['VILA VELHA', 'VITÓRIA', 'CARIACICA', 'SERRA', 'VIANA']
dfgv_cconfirmados = df_confirmados.query("Municipio == @gv")
dist_conf_gv = dfgv_cconfirmados.groupby('DataNotificacao').count().DataCadastro
rm_conf_gv = dist_conf_gv.rolling(14).mean()

# Análise dos mortos na GV

df_ob_gv = df_obitos.query("Municipio == @gv")
dist_ob_gv = df_ob_gv.groupby('DataObito').count().DataNotificacao
rm_dist_ob_gv = dist_ob_gv.rolling(14).mean()

In [ ]:
fig = px.line(x=["a","b","c"], y=[1,3,2], title="sample figure")
fig.show()

In [ ]:
df_conf = pd.merge(dist_confirmados, rm_confirmados, left_index=True, right_index=True)
df_conf

In [ ]:
fig2 = go.Figure()
fig2 = px.bar(df_conf) 
            #labels={'DataNotificacao':'Data de Notificação do caso','value':'Pacientes confirmados'})
fig2.update_layout(legend=dict(y=0.99, x=0.01))

fig2.show()

In [ ]:
fig3 = go.Figure(layout_title_text="Casos confirmados de COVID 19 no ES")

fig3.add_bar(x=dist_confirmados.index, y=dist_confirmados, name='Casos por dia')
fig3.add_scatter(x=rm_confirmados.index, y=rm_confirmados, name='Média móvel 14d')

fig3.update_layout(legend=dict(y=0.99, x=0.01))
fig3.update_traces(hovertemplate='Data: %{x}<br>Casos por dia: %{y}', selector={'name':'Casos por dia'})
fig3.update_traces(hovertemplate='Data: %{x}<br>Média móvel no dia: %{y}', selector={'name':'Média móvel 14d'})
fig3.update_yaxes(title='Pacientes confirmados')
fig3.update_xaxes(title='Data de Notificação')

fig3.show()

In [ ]:
fig4 = go.Figure(layout_title_text="Casos confirmados de COVID 19 na GV")

fig4.add_bar(x=dist_conf_gv.index, y=dist_conf_gv, name='Casos por dia')
fig4.add_scatter(x=rm_conf_gv.index, y=rm_conf_gv, name='Média móvel 14d')

fig4.update_layout(legend=dict(y=0.99, x=0.01))
fig4.update_traces(hovertemplate='Data: %{x}<br>Casos por dia: %{y}', selector={'name':'Casos por dia'})
fig4.update_traces(hovertemplate='Data: %{x}<br>Média móvel no dia: %{y}', selector={'name':'Média móvel 14d'})
fig4.update_yaxes(title='Pacientes confirmados')
fig4.update_xaxes(title='Data de Notificação')

fig4.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                     dict(step="all")
                     ])),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig4.show()

In [ ]:
fig5 = go.Figure(layout_title_text="Óbitos de COVID 19 na GV")

fig5.add_bar(x=dist_ob_gv.index, y=dist_ob_gv, name='Óbitos por dia')
fig5.add_scatter(x=rm_dist_ob_gv.index, y=rm_dist_ob_gv, name='Média móvel 14d')

fig5.update_layout(legend=dict(y=0.99, x=0.01))
fig5.update_traces(hovertemplate='Data: %{x}<br>Óbitos por dia: %{y}', selector={'name':'Óbitos por dia'})
fig5.update_traces(hovertemplate='Data: %{x}<br>Média móvel no dia: %{y}', selector={'name':'Média móvel 14d'})
fig5.update_yaxes(title='Pacientes mortos')

fig5.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                     dict(step="all")
                     ])),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig5.show()

## Tentando utilizar a interatividade a partir de sliders e botões

In [ ]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 5, 0.1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 10, 0.01),
            y=np.sin(step * np.arange(0, 10, 0.01))))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
